In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
import os
import re

In [ ]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [ ]:
query = "SELECT * FROM flat_data where rating_coding = '';"

with engine.connect() as conn:
    df = pd.read_sql(query, conn)

In [ ]:
K = len(df)
similarity = np.empty((K,K), dtype=float)

for i, ac in enumerate(df['table_content']):
    for j, bc in enumerate(df['table_content']):
        if i > j:
            continue
        if i == j:
            sim = 100
        else:
            sim = fuzz.ratio(ac, bc) # Use whatever metric you want here
                                     # for comparison of 2 strings.
        similarity[i, j] = sim
        similarity[j, i] = sim

In [ ]:
df_sim = pd.DataFrame(similarity, index=df.tableId, columns=df.tableId)
df_sim.to_csv('tableContent_similarityScorev1.csv')

In [4]:
df1 = pd.read_csv('tableContent_similarityScorev1.csv')

c = df1.corr().abs()

s = c.unstack()
so = s.sort_values(kind="quicksort")
so.to_csv('similarityScores_sorted.csv')